In [2]:
!pip install pyodbc
!pip install sqlalchemy


ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [1]:
import sqlite3
import pandas as pd
import numpy as np
import pyodbc
from sqlalchemy import create_engine

In [4]:
pd.set_option('display.width', 140)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 100)
pd.options.display.width
pd.options.display.max_rows

10000

In [6]:
dbconn = sqlite3.connect('source.db')
shippers_extract = pd.read_sql_query("SELECT * FROM Shippers", dbconn)
shippers_extract
dbconn.close()

In [7]:
dwconn = sqlite3.connect('datawarehouse.db')
c = dwconn.cursor()

In [8]:
delete_sshippers = c.execute("DELETE FROM S_Shippers")
#Insert data into S_Shippers
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-101', 'Speedy International', '(503) 555-9831')''')
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-102', 'United Package', '(503) 555-8712')''')
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-103', 'Federal Shipping', '(503) 555-9931')''')
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-104', 'Lazaro Bulk Corporation', '(503) 555-2399')''')
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-105', 'Pacia States' , '(503) 555-5564')''')
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-106', 'We Like to Move it', '(503) 555-1234')''')
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-107', 'Transpo Updates', '(503) 555-6520')''')
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-107', 'Transpo Updates', '(503) 555-6520')''')
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-108', 'Log Logistics', '(503) 555-2374')''')
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-109', 'Fast and the Slow', '(503) 555-7712')''')
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-110', 'Tangera Express', '(503) 555-2334')''')
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-111', 'NULL', '(503) 555-2334')''')
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-112', 'IDLF Logistics',NULL)''')
c.execute('''INSERT INTO S_Shippers (ShipperID, CompanyName, Phone) VALUES ('ERLJ-113', 'United Package', '(503) 555-8712')''')
c.execute('SELECT * FROM S_Shippers')

In [12]:
c.fetchall()
#why fetchall() is not working?
dwconn.commit()


In [13]:
c.fetchall()


[]

In [14]:
#its empty because fetchall() is not working
dwconn.commit()

In [18]:
s_table_new_data_df = pd.read_sql('''SELECT * FROM S_Shippers WHERE ShipperID NOT IN (SELECT ShipperID FROM M_Shippers)''', dwconn)
S_table_changed_company_name_df = pd.read_sql('''SELECT  s.ShipperID, s.CompanyName, s.Phone
                                               FROM S_Shippers s INNER JOIN M_Shippers m ON s.ShipperID = m.ShipperID
                                               WHERE NOT s.CompanyName = m.CompanyName''', dwconn)
S_table_changed_phone_number_df = pd.read_sql('''SELECT s.ShipperID, s.CompanyName, s.Phone FROM S_Shippers s 
                                              INNER JOIN M_Shippers m ON s.ShipperID = m.ShipperID 
                                              WHERE NOT s.Phone = m.Phone''', dwconn)
s_table_changed_data_df = S_table_changed_company_name_df.append(S_table_changed_phone_number_df, ignore_index=True)
s_table_changed_data_df
s_table_extract_df = s_table_new_data_df.append(s_table_changed_data_df, ignore_index=True)
s_table_extract_df


C:\Users\Admin\AppData\Local\Temp\ipykernel_12456\3050788986.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  s_table_changed_data_df = S_table_changed_company_name_df.append(S_table_changed_phone_number_df, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_12456\3050788986.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  s_table_extract_df = s_table_new_data_df.append(s_table_changed_data_df, ignore_index=True)


,ShipperID,CompanyName,Phone
0,ERLJ-108,Log Logistics,(503) 555-2374
1,ERLJ-109,Fast and the Slow,(503) 555-7712
2,ERLJ-110,Tangera Express,(503) 555-2334
3,ERLJ-111,NULL,(503) 555-2334
4,ERLJ-112,IDLF Logistics,None
5,ERLJ-113,United Package,(503) 555-8712
6,ERLJ-101,Speedy International,(503) 555-9831
7,ERLJ-105,Pacia States,(503) 555-5564
8,ERLJ-104,Lazaro Bulk Corporation,(503) 555-2399


In [19]:
delete_xshipper = c.execute("DELETE FROM X_Shippers")
dwconn.commit()
c.execute('SELECT * FROM X_Shippers')
c.fetchall()

[]

In [22]:
cols = "`,`".join([str(i) for i in s_table_extract_df.columns.tolist()])
value = None

for i,row in s_table_extract_df.iterrows():
    sql = "INSERT INTO `X_Shippers` (`" +cols + "`) VALUES (" + "'%s',"*(len(row)-1) + "'%s" + "')"
    a_string = sql %tuple(row) if value is None else value
    c.execute(a_string)
    dwconn.commit()
c.execute('''SELECT * FROM X_Shippers''')
c.fetchall()

[('ERLJ-108', 'Log Logistics', '(503) 555-2374'),
 ('ERLJ-109', 'Fast and the Slow', '(503) 555-7712'),
 ('ERLJ-110', 'Tangera Express', '(503) 555-2334'),
 ('ERLJ-111', 'NULL', '(503) 555-2334'),
 ('ERLJ-112', 'IDLF Logistics', 'None'),
 ('ERLJ-113', 'United Package', '(503) 555-8712'),
 ('ERLJ-101', 'Speedy International', '(503) 555-9831'),
 ('ERLJ-105', 'Pacia States', '(503) 555-5564'),
 ('ERLJ-104', 'Lazaro Bulk Corporation', '(503) 555-2399')]

In [24]:
#%%Clean x table and insert into error table
#select campanies with null names
x_table_no_companyname_df =pd.read_sql('''SELECT * FROM X_Shippers WHERE CompanyName = 'None' ''', dwconn)
x_table_no_companyname_df['ErrorType'] = 'No Company Name'
x_table_duplicate_companies_df = pd.read_sql('''SELECT * FROM X_Shippers 
                                           WHERE CompanyName IN 
                                           (SELECT CompanyName FROM S_Shippers 
                                           GROUP BY CompanyName HAVING COUNT(CompanyName) > 1)''', dwconn)
x_table_duplicate_companies_df['ErrorType'] = 'Duplicate Company Name'  
x_table_errors_df = pd.concat([x_table_no_companyname_df, x_table_duplicate_companies_df])
#Set unknown to Missing Phone Numbers
update_xshippers = c.execute('''UPDATE X_Shippers SET Phone = 'Unknown Phone Number' WHERE Phone = 'None' ''')
c.execute("SELECT * FROM X_Shippers")
c.fetchall()
#Insert  unknown phone numbers into error table(E table)
#Delete data inside E_Shippers first
delete_eshippers = c.execute("DELETE FROM E_Shippers")
c.execute("SELECT * FROM E_Shippers")
c.fetchall()


[]

In [26]:
#Creating Column list for insertion
cols = "`,`".join([str(i) for i in x_table_errors_df.columns.tolist()])
#insert records one by one into E_Shippers
for i,row in x_table_errors_df.iterrows():
    sql = "INSERT INTO `E_Shippers` (`" +cols + "`) VALUES (" + "'%s',"*(len(row)-1) + "'%s" + "')"
    a_string = sql %tuple(row) if value is None else value
    c.execute(a_string)
    dwconn.commit()
c.execute('''SELECT * FROM E_Shippers''')
c.fetchall()

[('ERLJ-113', 'United Package', '(503) 555-8712', 'Duplicate Company Name')]

In [27]:
x_table_clean_data_df = pd.read_sql('''SELECT * FROM X_Shippers WHERE ShipperID NOT IN (SELECT ShipperID FROM E_Shippers)''', dwconn)
delete_cshipper = c.execute("DELETE FROM C_Shippers")
c.execute("SELECT * FROM C_Shippers")
c.fetchall()

[]

In [28]:
#actual insert into C table
#creating column list for insertion
cols = "`,`".join([str(i) for i in x_table_clean_data_df.columns.tolist()])

#insert records one by one into C_Shippers
for i,row in x_table_clean_data_df.iterrows():
    sql = "INSERT INTO `C_Shippers` (`" +cols + "`) VALUES (" + "'%s',"*(len(row)-1) + "'%s" + "')"
    a_string = sql %tuple(row) if value is None else value
    c.execute(a_string)
    dwconn.commit()
#check if inserted
c.execute('''SELECT * FROM C_Shippers''')
c.fetchall()

[('ERLJ-108', 'Log Logistics', '(503) 555-2374'),
 ('ERLJ-109', 'Fast and the Slow', '(503) 555-7712'),
 ('ERLJ-110', 'Tangera Express', '(503) 555-2334'),
 ('ERLJ-111', 'NULL', '(503) 555-2334'),
 ('ERLJ-112', 'IDLF Logistics', 'Unknown Phone Number'),
 ('ERLJ-101', 'Speedy International', '(503) 555-9831'),
 ('ERLJ-105', 'Pacia States', '(503) 555-5564'),
 ('ERLJ-104', 'Lazaro Bulk Corporation', '(503) 555-2399')]

In [30]:
#%%Update M Table
#select all data from c tables
c_table_new_date_df = pd.read_sql('''SELECT * FROM C_Shippers c
                                  WHERE c.ShipperID NOT IN 
                                  (SELECT m.ShipperID FROM M_Shippers m )''', dwconn)
#INSERT Clean data into m table
#creating column list for insertion
cols = "`,`".join([str(i) for i in c_table_new_date_df.columns.tolist()])
#insert records one by one into M_Shippers
for i,row in c_table_new_date_df.iterrows():
    sql = "INSERT INTO M_Shippers_Test (`" +cols + "`) VALUES (" + "'%s',"*(len(row)-1) + "'%s" + "')"
    a_string = sql %tuple(row) if value is None else value
    c.execute(a_string)
    dwconn.commit()
#check if inserted
c.execute('''SELECT * FROM M_Shippers_Test''')
c.fetchall()

[('ERLJ-101', 'Speedy Express', '(503) 555-9831'),
 ('ERLJ-102', 'United Package', '(503) 555-8712'),
 ('ERLJ-103', 'Federal Shipping', '(503) 555-9931'),
 ('ERLJ-104', 'Lazaro Bulk Corporation', '(503) 555-2388'),
 ('ERLJ-105', 'Pacia Estates', '(503) 555-5564'),
 ('ERLJ-106', 'We Like to Move it', '(503) 555-1234'),
 ('ERLJ-107', 'Transpo Updates', '(503) 555-6520'),
 ('ERLJ-108', 'Log Logistics', '(503) 555-2374'),
 ('ERLJ-109', 'Fast and the Slow', '(503) 555-7712'),
 ('ERLJ-110', 'Tangera Express', '(503) 555-2334'),
 ('ERLJ-111', 'NULL', '(503) 555-2334'),
 ('ERLJ-112', 'IDLF Logistics', 'Unknown Phone Number')]

In [32]:
#%% Processing Changed data and Update the master Data
#select all data from c tables
c_table_changed_data_df = pd.read_sql('''SELECT c.* 
                                      FROM C_Shippers c, M_Shippers m 
                                      WHERE c.ShipperID = m.ShipperID 
                                      AND (c.CompanyName <> m.CompanyName OR c.Phone <> m.Phone)''', dwconn)
delete_mshipperstest = c.execute('''DELETE FROM M_Shippers_Test
                                 WHERE ShipperID IN (SELECT m.ShipperID FROM c_Shippers c, M_Shippers m
                                 WHERE c.ShipperID = m.ShipperID
                                 AND (c.CompanyName <> m.CompanyName OR c.Phone <> m.Phone))''')
c.execute('''SELECT * FROM M_Shippers_Test''')
c.fetchall()
dwconn.commit()

In [33]:
c_table_changed_data_df

,ShipperID,CompanyName,Phone
0,ERLJ-101,Speedy International,(503) 555-9831
1,ERLJ-105,Pacia States,(503) 555-5564
2,ERLJ-104,Lazaro Bulk Corporation,(503) 555-2399


In [34]:
#insert clean data into m table with changed data
#creating column list for insertion
cols = "`,`".join([str(i) for i in c_table_changed_data_df.columns.tolist()])
#insert records one by one into M_Shippers
for i,row in c_table_changed_data_df.iterrows():
    sql = "INSERT INTO M_Shippers_Test (`" +cols + "`) VALUES (" + "'%s',"*(len(row)-1) + "'%s" + "')"
    a_string = sql %tuple(row) if value is None else value
    c.execute(a_string)
    dwconn.commit()
c.execute('''SELECT * FROM M_Shippers_Test''')
c.fetchall()


[('ERLJ-102', 'United Package', '(503) 555-8712'),
 ('ERLJ-103', 'Federal Shipping', '(503) 555-9931'),
 ('ERLJ-106', 'We Like to Move it', '(503) 555-1234'),
 ('ERLJ-107', 'Transpo Updates', '(503) 555-6520'),
 ('ERLJ-108', 'Log Logistics', '(503) 555-2374'),
 ('ERLJ-109', 'Fast and the Slow', '(503) 555-7712'),
 ('ERLJ-110', 'Tangera Express', '(503) 555-2334'),
 ('ERLJ-111', 'NULL', '(503) 555-2334'),
 ('ERLJ-112', 'IDLF Logistics', 'Unknown Phone Number'),
 ('ERLJ-101', 'Speedy International', '(503) 555-9831'),
 ('ERLJ-105', 'Pacia States', '(503) 555-5564'),
 ('ERLJ-104', 'Lazaro Bulk Corporation', '(503) 555-2399')]

In [36]:
#%% Transform processess
#select data from c and transform to DW_Format
c_table_data_df = pd.read_sql('''SELECT ShipperID as [Shipper_ID], CompanyName as [Shipper_Name], Phone as [Current_Shipper_Phone],
                              DATE() as [Effective_Date] FROM C_Shippers''', dwconn)
c_table_data_df['Previous_Shipper_Phone'] = "Previous_Shipper_Phone"
c_table_data_df = c_table_data_df[['Shipper_ID', 'Shipper_Name', 'Current_Shipper_Phone', 'Previous_Shipper_Phone', 'Effective_Date']]


In [40]:
#%% INSERT INTO T TABLE
#delete existing data in T table
delete_cshippers = c.execute("DELETE FROM T_Shipper")
c.execute("SELECT * FROM T_Shipper")
c.fetchall()

[]

In [41]:
#%%Actual insert into c table into T table(c_shipper into t_shipper)
#creating column list for insertion
cols = "`,`".join([str(i) for i in c_table_data_df.columns.tolist()])
#insert records one by one into T_Shippers
for i,row in c_table_data_df.iterrows():
    sql = "INSERT INTO `T_Shipper` (`" +cols + "`) VALUES (" + "'%s',"*(len(row)-1) + "'%s" + "')"
    a_string = sql %tuple(row) if value is None else value
    c.execute(a_string)
    dwconn.commit()
pd.read_sql('''SELECT * FROM T_Shipper''', dwconn)

,Shipper_ID,Shipper_Name,Current_Shipper_Phone,Previous_Shipper_Phone,Effective_Date
0,ERLJ-108,Log Logistics,(503) 555-2374,Previous_Shipper_Phone,2023-09-24
1,ERLJ-109,Fast and the Slow,(503) 555-7712,Previous_Shipper_Phone,2023-09-24
2,ERLJ-110,Tangera Express,(503) 555-2334,Previous_Shipper_Phone,2023-09-24
3,ERLJ-111,NULL,(503) 555-2334,Previous_Shipper_Phone,2023-09-24
4,ERLJ-112,IDLF Logistics,Unknown Phone Number,Previous_Shipper_Phone,2023-09-24
5,ERLJ-101,Speedy International,(503) 555-9831,Previous_Shipper_Phone,2023-09-24
6,ERLJ-105,Pacia States,(503) 555-5564,Previous_Shipper_Phone,2023-09-24
7,ERLJ-104,Lazaro Bulk Corporation,(503) 555-2399,Previous_Shipper_Phone,2023-09-24


In [43]:
#%% SELECT  new Data from the t table 
t_table_new_data_df = pd.read_sql('''SELECT t.*
                                  FROM t_Shipper t
                                  LEFT JOIN d_Shipper d ON t.Shipper_ID = d.Shipper_ID
                                  WHERE d.Shipper_ID IS NULL''', dwconn)
t_table_new_data_df['Current_Row_Indicator'] = 'Y'

In [45]:
#%% INSERT new data into i table
#delete existing data in I table
delete_ishippers = c.execute("DELETE FROM I_Shipper")
c.execute("SELECT * FROM I_Shipper")
c.fetchall()

# add missing column to I_Shipper table
c.execute("ALTER TABLE I_Shipper ADD COLUMN Current_Row_Indicator TEXT")

#creating column list for insertion
cols = "`,`".join([str(i) for i in t_table_new_data_df.columns.tolist()])
#insert records one by one into I_Shippers
for i,row in t_table_new_data_df.iterrows():
    sql = "INSERT INTO `I_Shipper` (`" +cols + "`, `Current_Row_Indicator`) VALUES (" + "'%s',"*(len(row)-1) + "'%s', 'Y')"
    a_string = sql %tuple(row) if value is None else value
    c.execute(a_string)
    dwconn.commit()


In [46]:
pd.read_sql('''SELECT * FROM I_Shipper''', dwconn)

,Shipper_ID,Shipper_Name,Current_Shipper_Phone,Previous_Shipper_Phone,Effective_Date,Current_Row_Ind,Current_Row_Indicator
0,ERLJ-108,Log Logistics,(503) 555-2374,Previous_Shipper_Phone,2023-09-24,None,Y
1,ERLJ-109,Fast and the Slow,(503) 555-7712,Previous_Shipper_Phone,2023-09-24,None,Y
2,ERLJ-110,Tangera Express,(503) 555-2334,Previous_Shipper_Phone,2023-09-24,None,Y
3,ERLJ-111,NULL,(503) 555-2334,Previous_Shipper_Phone,2023-09-24,None,Y
4,ERLJ-112,IDLF Logistics,Unknown Phone Number,Previous_Shipper_Phone,2023-09-24,None,Y


In [47]:
#%% Insert changed data into u table
#select changed data from t table
t_table_changed_data_df = pd.read_sql('''SELECT t.*
                                      FROM t_Shipper t
                                      INNER JOIN d_Shipper d ON t.Shipper_ID = d.Shipper_ID
                                      WHERE (NOT t.Shipper_Name = d.Shipper_Name or NOT 
                                            t.Current_Shipper_Phone = d.Current_Shipper_Phone)
                                      AND d.Current_Row_Ind IN ('Y')''', dwconn)
t_table_changed_data_df['Current_Row_Ind'] = 'Y'
#delete existing data in U table first
delete_ushippers = c.execute("DELETE FROM U_Shipper")
c.execute("SELECT * FROM U_Shipper")
c.fetchall()

[]

In [48]:
#%%Actual insert of changed data into U table
#insert changed data into U
#creating column list for insertion
cols = "`,`".join([str(i) for i in t_table_changed_data_df.columns.tolist()])
#insert records one by one into U_Shippers
for i,row in t_table_changed_data_df.iterrows():
    sql = "INSERT INTO `U_Shipper` (`" +cols + "`) VALUES (" + "'%s',"*(len(row)-1) + "'%s')"
    a_string = sql %tuple(row) if value is None else value
    c.execute(a_string)
    dwconn.commit()
pd.read_sql('''SELECT * FROM U_Shipper''', dwconn)

,Shipper_ID,Shipper_Name,Current_Shipper_Phone,Previous_Shipper_Phone,Effective_Date,Current_Row_Ind
0,ERLJ-101,Speedy International,(503) 555-9831,Previous_Shipper_Phone,2023-09-24,Y
1,ERLJ-104,Lazaro Bulk Corporation,(503) 555-2399,Previous_Shipper_Phone,2023-09-24,Y
2,ERLJ-105,Pacia States,(503) 555-5564,Previous_Shipper_Phone,2023-09-24,Y


In [49]:
#%% Insert  I INTO D Table 
#Get Max Warehouse key
maxkey = pd.read_sql('''SELECT MAX(Shipper_Key) as MAX FROM D_Shipper''', dwconn)
#select data to be inserted into I table
i_table_data_df = pd.read_sql('''SELECT * FROM I_Shipper''', dwconn)
#identify the next set of shipper_key's to be assigned to the new data from i table
i_table_data_df['Shipper_Key'] = np.arange(pd.to_numeric(maxkey.iloc[0]).values+1,
                                           (pd.to_numeric(maxkey.iloc[0].values)+len(i_table_data_df)+1))
#rearrange according to the d table format of columns
i_table_data_df = i_table_data_df[['Shipper_Key', 'Shipper_ID', 'Shipper_Name', 'Current_Shipper_Phone', 'Previous_Shipper_Phone', 'Effective_Date', 'Current_Row_Ind']]


In [50]:
i_table_data_df

,Shipper_Key,Shipper_ID,Shipper_Name,Current_Shipper_Phone,Previous_Shipper_Phone,Effective_Date,Current_Row_Ind
0,8,ERLJ-108,Log Logistics,(503) 555-2374,Previous_Shipper_Phone,2023-09-24,None
1,9,ERLJ-109,Fast and the Slow,(503) 555-7712,Previous_Shipper_Phone,2023-09-24,None
2,10,ERLJ-110,Tangera Express,(503) 555-2334,Previous_Shipper_Phone,2023-09-24,None
3,11,ERLJ-111,NULL,(503) 555-2334,Previous_Shipper_Phone,2023-09-24,None
4,12,ERLJ-112,IDLF Logistics,Unknown Phone Number,Previous_Shipper_Phone,2023-09-24,None


In [51]:
#now insert into D table
#creating column list for insertion
cols = "`,`".join([str(i) for i in i_table_data_df.columns.tolist()])
#insert records one by one into D_Shippers
for i,row in i_table_data_df.iterrows():
    sql = "INSERT INTO `D_Shipper` (`" +cols + "`) VALUES (" + "'%s',"*(len(row)-1) + "'%s')"
    a_string = sql %tuple(row) if value is None else value
    c.execute(a_string)
    dwconn.commit()
pd.read_sql('''SELECT * FROM D_Shipper''', dwconn)


,Shipper_Key,Shipper_ID,Shipper_Name,Current_Shipper_Phone,Previous_Shipper_Phone,Effective_Date,Current_Row_Ind
0,1,ERLJ-101,Speedy Express,(503) 555-9831,Previous Phone Unknown,10/31/2014,Y
1,2,ERLJ-102,United Package,(503) 555-8712,Previous Phone Unknown,10/31/2014,Y
2,3,ERLJ-103,Federal Shipping,(503) 555-9931,Previous Phone Unknown,10/31/2014,Y
3,4,ERLJ-104,Lazaro Bulk Corporation,(503) 555-2388,Previous Phone Unknown,10/31/2014,Y
4,5,ERLJ-105,Pacia Estates,(503) 555-5564,Previous Phone Unknown,10/31/2014,Y
5,6,ERLJ-106,We Like to Move it,(503) 555-1234,Previous Phone Unknown,10/31/2014,Y
6,7,ERLJ-107,Transpo Updates,(503) 555-6520,Previous Phone Unknown,10/31/2014,Y
7,8,ERLJ-108,Log Logistics,(503) 555-2374,Previous_Shipper_Phone,2023-09-24,None
8,9,ERLJ-109,Fast and the Slow,(503) 555-7712,Previous_Shipper_Phone,2023-09-24,None
9,10,ERLJ-110,Tangera Express,(503) 555-2334,Previous_Shipper_Phone,2023-09-24,None


In [63]:
#%%the update process needs to check different types of history tracking 
#for dimensions . in this case ,we only have type 3 and type 2
#select type 3 from u table and then update the d table
u_table_type3_data_df = pd.read_sql('''SELECT u.*
                                    FROM U_Shipper u
                                    INNER JOIN D_Shipper d ON u.Shipper_ID = d.Shipper_ID
                                    WHERE NOT (u.Current_Shipper_Phone = d.Current_Shipper_Phone)
                                    AND d.Current_Row_Ind IN ('Y')''', dwconn)
d_table_type3_data_df = pd.read_sql('''SELECT d.*
                                    FROM u_Shipper u INNER JOIN D_Shipper d ON u.Shipper_ID = d.Shipper_ID
                                    WHERE NOT (u.Current_Shipper_Phone = d.Current_Shipper_Phone)
                                    AND d.Current_Row_Ind IN ('Y')''', dwconn)
u_table_type3_data_df['Previous_Shipper_Phone'] = d_table_type3_data_df['Current_Shipper_Phone']
u_table_type3_data_df['Shipper_Key'] = d_table_type3_data_df['Shipper_Key']
u_table_type3_data_df = u_table_type3_data_df[['Shipper_Key', 'Shipper_ID', 'Shipper_Name', 'Current_Shipper_Phone', 'Previous_Shipper_Phone', 'Effective_Date', 'Current_Row_Ind']]

pd.read_sql('''SELECT * FROM D_Shipper''', dwconn)


,Shipper_Key,Shipper_ID,Shipper_Name,Current_Shipper_Phone,Previous_Shipper_Phone,Effective_Date,Current_Row_Ind
0,1,ERLJ-101,Speedy Express,(503) 555-9831,Previous Phone Unknown,10/31/2014,Y
1,2,ERLJ-102,United Package,(503) 555-8712,Previous Phone Unknown,10/31/2014,Y
2,3,ERLJ-103,Federal Shipping,(503) 555-9931,Previous Phone Unknown,10/31/2014,Y
3,4,ERLJ-104,Lazaro Bulk Corporation,(503) 555-2388,Previous Phone Unknown,10/31/2014,Y
4,5,ERLJ-105,Pacia Estates,(503) 555-5564,Previous Phone Unknown,10/31/2014,Y
5,6,ERLJ-106,We Like to Move it,(503) 555-1234,Previous Phone Unknown,10/31/2014,Y
6,7,ERLJ-107,Transpo Updates,(503) 555-6520,Previous Phone Unknown,10/31/2014,Y
7,8,ERLJ-108,Log Logistics,(503) 555-2374,Previous_Shipper_Phone,2023-09-24,None
8,9,ERLJ-109,Fast and the Slow,(503) 555-7712,Previous_Shipper_Phone,2023-09-24,None
9,10,ERLJ-110,Tangera Express,(503) 555-2334,Previous_Shipper_Phone,2023-09-24,None


In [66]:
cols = "`,`".join([str(i) for i in u_table_type3_data_df.columns.tolist()])
#Insert records from the u table one by one into the d table
for i,row in u_table_type3_data_df.iterrows():
    # check if the record already exists in the D_Shipper table
    existing_record = pd.read_sql(f"SELECT * FROM D_Shipper WHERE Shipper_Key = {row['Shipper_Key']}", dwconn)
    if not existing_record.empty:
        # update the existing record
        c.execute(f"UPDATE D_Shipper SET Shipper_Name = '{row['Shipper_Name']}', Current_Shipper_Phone = '{row['Current_Shipper_Phone']}', Previous_Shipper_Phone = '{row['Previous_Shipper_Phone']}', Effective_Date = '{row['Effective_Date']}' WHERE Shipper_Key = {row['Shipper_Key']}")
    else:
        # insert a new record
        sql = "INSERT INTO `D_Shipper` (`" +cols + "`) VALUES (" + "'%s',"*(len(row)-1) + "'%s')"
        a_string = sql %tuple(row) if value is None else value
        c.execute(a_string)
    dwconn.commit()


In [67]:
pd.read_sql('''SELECT * FROM D_Shipper''', dwconn)

,Shipper_Key,Shipper_ID,Shipper_Name,Current_Shipper_Phone,Previous_Shipper_Phone,Effective_Date,Current_Row_Ind
0,1,ERLJ-101,Speedy Express,(503) 555-9831,Previous Phone Unknown,10/31/2014,Y
1,2,ERLJ-102,United Package,(503) 555-8712,Previous Phone Unknown,10/31/2014,Y
2,3,ERLJ-103,Federal Shipping,(503) 555-9931,Previous Phone Unknown,10/31/2014,Y
3,4,ERLJ-104,Lazaro Bulk Corporation,(503) 555-2399,(503) 555-2388,2023-09-24,Y
4,5,ERLJ-105,Pacia Estates,(503) 555-5564,Previous Phone Unknown,10/31/2014,Y
5,6,ERLJ-106,We Like to Move it,(503) 555-1234,Previous Phone Unknown,10/31/2014,Y
6,7,ERLJ-107,Transpo Updates,(503) 555-6520,Previous Phone Unknown,10/31/2014,Y
7,8,ERLJ-108,Log Logistics,(503) 555-2374,Previous_Shipper_Phone,2023-09-24,None
8,9,ERLJ-109,Fast and the Slow,(503) 555-7712,Previous_Shipper_Phone,2023-09-24,None
9,10,ERLJ-110,Tangera Express,(503) 555-2334,Previous_Shipper_Phone,2023-09-24,None


In [68]:
maxkey = pd.read_sql('''SELECT MAX(Shipper_Key) as MAX FROM D_Shipper''', dwconn)
#select data to be inserted into u table
u_table_type2_data_df = pd.read_sql('''SELECT u.*
                                    FROM u_shipper u
                                    INNER JOIN d_shipper d ON u.Shipper_ID = d.Shipper_ID
                                    WHERE NOT (u.Shipper_Name = d.Shipper_Name)
                                    AND d.Current_Row_Ind IN ('Y')''', dwconn)
u_table_type2_data_df['Shipper_Key'] = np.arange(pd.to_numeric(maxkey.iloc[0]).values+1,
                                             (pd.to_numeric(maxkey.iloc[0].values)+len(u_table_type2_data_df)+1))
u_table_type2_data_df = u_table_type2_data_df[['Shipper_Key', 'Shipper_ID', 'Shipper_Name', 'Current_Shipper_Phone', 'Previous_Shipper_Phone', 'Effective_Date', 'Current_Row_Ind']]


In [69]:
#now insert changed data (type 2) from u table into d table
#creating column list for insertion
cols = "`,`".join([str(i) for i in u_table_type2_data_df.columns.tolist()])
#insert records from the u table one by one into the d table
for i,row in u_table_type2_data_df.iterrows():
    sql = "INSERT INTO D_Shipper (`" +cols + "`) VALUES (" + "'%s',"*(len(row)-1) + "'%s" + "')"
    a_string = sql %tuple(row) if value is None else value
    c.execute(a_string)
    dwconn.commit()
pd.read_sql('''SELECT * FROM D_Shipper''', dwconn)

,Shipper_Key,Shipper_ID,Shipper_Name,Current_Shipper_Phone,Previous_Shipper_Phone,Effective_Date,Current_Row_Ind
0,1,ERLJ-101,Speedy Express,(503) 555-9831,Previous Phone Unknown,10/31/2014,Y
1,2,ERLJ-102,United Package,(503) 555-8712,Previous Phone Unknown,10/31/2014,Y
2,3,ERLJ-103,Federal Shipping,(503) 555-9931,Previous Phone Unknown,10/31/2014,Y
3,4,ERLJ-104,Lazaro Bulk Corporation,(503) 555-2399,(503) 555-2388,2023-09-24,Y
4,5,ERLJ-105,Pacia Estates,(503) 555-5564,Previous Phone Unknown,10/31/2014,Y
5,6,ERLJ-106,We Like to Move it,(503) 555-1234,Previous Phone Unknown,10/31/2014,Y
6,7,ERLJ-107,Transpo Updates,(503) 555-6520,Previous Phone Unknown,10/31/2014,Y
7,8,ERLJ-108,Log Logistics,(503) 555-2374,Previous_Shipper_Phone,2023-09-24,None
8,9,ERLJ-109,Fast and the Slow,(503) 555-7712,Previous_Shipper_Phone,2023-09-24,None
9,10,ERLJ-110,Tangera Express,(503) 555-2334,Previous_Shipper_Phone,2023-09-24,None


In [71]:
#%% Update Current Indicators in d table 
#update shipper Current Value
for_update_data_df = pd.read_sql('''SELECT d.*
                            FROM u_shipper u INNER JOIN d_shipper d ON u.Shipper_ID = d.Shipper_ID
                            WHERE d.Current_Row_Ind = 'Y'
                            AND NOT u.Shipper_Name = d.Shipper_Name''', dwconn)
for_update_data_df['Current_Row_Ind'] = 'N'
#delete from d shipper first
delete_dshippers = c.execute('''DELETE FROM d_shipper
                                WHERE Shipper_Key IN (SELECT d.Shipper_Key
                                                      FROM u_shipper u INNER JOIN d_shipper d ON u.Shipper_ID = d.Shipper_ID
                                                      WHERE d.Current_Row_Ind = 'Y'
                                                      AND NOT u.Shipper_Name = d.Shipper_Name)''')


In [72]:
#actual update of current row indicator
#creating column list for insertion
cols = "`,`".join([str(i) for i in for_update_data_df.columns.tolist()])
#insert update records one by one into D_Shipper
for i,row in for_update_data_df.iterrows():
    sql = "INSERT INTO D_Shipper (`" +cols + "`) VALUES (" + "'%s',"*(len(row)-1) + "'%s"+ "')"
    a_string = sql %tuple(row) if value is None else value
    c.execute(a_string)
    dwconn.commit()

In [73]:
pd.read_sql('''SELECT * FROM D_Shipper''', dwconn)

,Shipper_Key,Shipper_ID,Shipper_Name,Current_Shipper_Phone,Previous_Shipper_Phone,Effective_Date,Current_Row_Ind
0,1,ERLJ-101,Speedy Express,(503) 555-9831,Previous Phone Unknown,10/31/2014,N
1,2,ERLJ-102,United Package,(503) 555-8712,Previous Phone Unknown,10/31/2014,Y
2,3,ERLJ-103,Federal Shipping,(503) 555-9931,Previous Phone Unknown,10/31/2014,Y
3,4,ERLJ-104,Lazaro Bulk Corporation,(503) 555-2399,(503) 555-2388,2023-09-24,Y
4,5,ERLJ-105,Pacia Estates,(503) 555-5564,Previous Phone Unknown,10/31/2014,N
5,6,ERLJ-106,We Like to Move it,(503) 555-1234,Previous Phone Unknown,10/31/2014,Y
6,7,ERLJ-107,Transpo Updates,(503) 555-6520,Previous Phone Unknown,10/31/2014,Y
7,8,ERLJ-108,Log Logistics,(503) 555-2374,Previous_Shipper_Phone,2023-09-24,None
8,9,ERLJ-109,Fast and the Slow,(503) 555-7712,Previous_Shipper_Phone,2023-09-24,None
9,10,ERLJ-110,Tangera Express,(503) 555-2334,Previous_Shipper_Phone,2023-09-24,None
